# qmd

> Basic qmd generation helpers (experimental)
- order: 15

In [ ]:
#|hide
#|default_exp qmd

In [ ]:
#|export
from __future__ import annotations
from nbdev.config import *
from execnb.nbio import *
from pathlib import Path
import re

import sys,os,inspect

from fastcore.utils import *
from fastcore.meta import delegates

In [ ]:
#|hide
from fastcore.test import *

In [ ]:
#|export
def _qmd_to_raw_cell(source_str, cell_type_str, qmd_metadata=None):
    """Create a default ipynb json cell"""
    cell = {'cell_type': cell_type_str, 'metadata': {}, 'source': source_str}
    if cell_type_str == 'code':
        cell['execution_count'] = None
        cell['outputs'] = []
        if qmd_metadata: cell['qmd_metadata'] = qmd_metadata
    return cell

def read_qmd(path): 
    """Reads a .qmd file as an nb compatible with the rest of execnb and nbdev"""
    content = Path(path).read_text(encoding='utf-8')
    # Modified regex to capture the metadata between {python and }
    cell_pat = re.compile(r"^(`{3,})\s*\{python([^\}]*)\}\s*\n(.*?)^\1\s*$", re.MULTILINE | re.DOTALL)
    
    # `parts` will be [md_chunk, captured_backticks_1, captured_metadata_1, captured_code_1, md_chunk_2, ...]
    parts = cell_pat.split(content)
    raw_cells = []
    
    # Handle the first markdown segment (before any code cells, or all content if no code cells)
    initial_md_source = parts[0].strip()
    if initial_md_source: raw_cells.append(_qmd_to_raw_cell(initial_md_source, 'markdown'))
    
    # 4 items per match: [md, backticks, metadata?, code]
    for i in range(1, len(parts), 4):
        if i + 2 < len(parts):  # We have backticks, metadata, and code
            metadata = parts[i+1]  # The captured metadata
            code_source = parts[i+2].strip()  # The captured code
            if code_source: raw_cells.append(_qmd_to_raw_cell(code_source, 'code', metadata if metadata else None))
        if i + 3 < len(parts):  # Intermediate markdown
            intermediate_md_source = parts[i+3].strip()
            if intermediate_md_source: raw_cells.append(_qmd_to_raw_cell(intermediate_md_source, 'markdown'))
                
    # Construct the final notebook dictionary
    notebook_dict = {
        'cells': raw_cells,
        'metadata': {
            'kernelspec': {'display_name': 'Python 3', 'language': 'python', 'name': 'python3'},
            'language_info': {'name': 'python'},
            'path': str(path)
        },
        'nbformat': 4,
        'nbformat_minor': 5,
        'path_': str(path)
    }
    
    return dict2nb(notebook_dict)

def read_nb_or_qmd(path):
    if Path(path).suffix == '.qmd': return read_qmd(path)
    return read_nb(path)


In [ ]:
#| hide
# Test: we want the .qmd file to be very close to .ipynb file in the nb source (except for saved outputs which are not present in .qmd)
nb_qmd = read_qmd("../../tests/tst_index.qmd")
nb_ipynb = read_nb("../../tests/tst_index.ipynb")

In [ ]:
#| hide
tst_cell = """\
# Title 
> description

and a couple more pieces of information before the code cell:

``` {python .code-cell-2}
#| export
print(3+4)
```

```` {python .code-cell-2}
#| export
print(9+12)
````

`````{python}
print(5+6)
`````

```python
print("not a cell")
```
"""
cell_pat = re.compile(r"^(`{3,})\s*\{python[^\n]*\}\s*(.*?)^\1\s*$", re.MULTILINE | re.DOTALL)
matches = cell_pat.findall(tst_cell)
assert len(matches) == 3
assert len(matches[0][0]) == 3 # 3 backticks
assert len(matches[1][0]) == 4 # 4 backticks
assert len(matches[2][0]) == 5 # 5 backticks

assert matches[0][1] == '#| export\nprint(3+4)\n'
assert matches[1][1] == '#| export\nprint(9+12)\n'
assert matches[2][1] == 'print(5+6)\n'

for match in matches: print(match)


('```', '#| export\nprint(3+4)\n')
('````', '#| export\nprint(9+12)\n')
('`````', 'print(5+6)\n')


We similarly need a `write_qmd` function to write a notebook to a .qmd file.

In [ ]:
#|export
def _nb_to_qmd_str(nb):
    """Convert a notebook to a string in .qmd format"""
    def cell_to_qmd(cell):
        source = cell.source.rstrip('\n')
        if cell.cell_type in ['markdown', 'raw']: return source
        elif cell.cell_type == 'code':
            qmd_metadata = getattr(cell, 'qmd_metadata', None)
            if qmd_metadata: return f'```{{python{qmd_metadata}}}\n{source}\n```'
            else: return f'```{{python}}\n{source}\n```'
        return ''
    return '\n\n'.join(filter(None, [cell_to_qmd(cell) for cell in nb.cells]))

def write_qmd(nb, path):
    """Write a notebook back to .qmd format"""
    qmd_str = _nb_to_qmd_str(nb)
    Path(path).write_text(qmd_str, encoding='utf-8')
    
def write_nb_or_qmd(nb, path):
    if Path(path).suffix == '.qmd': write_qmd(nb, path)
    else: write_nb(nb, path)


In [ ]:
#| export
def meta(md,  # Markdown to add meta to
         classes=None,  # List of CSS classes to add
         style=None,  # Dict of CSS styles to add
         **kwargs):   # Additional attributes to add to meta
    "A metadata section for qmd div in `{}`"
    if style: kwargs['style'] = "; ".join(f'{k}: {v}' for k,v in style.items())
    props = ' '.join(f'{k}="{v}"' for k,v in kwargs.items())
    classes = ' '.join('.'+c for c in L(classes))
    meta = []
    if classes: meta.append(classes)
    if props: meta.append(props)
    meta = ' '.join(meta)
    return md + ("{" + meta + "}" if meta else "")

In [ ]:
#| export
def div(txt,  # Markdown to add meta to
        classes=None,  # List of CSS classes to add
        style=None,  # Dict of CSS styles to add
        **kwargs):
    "A qmd div with optional metadata section"
    return meta("::: ", classes=classes, style=style, **kwargs) + f"\n\n{txt}\n\n:::\n\n"

In [ ]:
#| export
def img(fname,  # Image to link to
        classes=None,  # List of CSS classes to add
        style=None,   # Dict of CSS styles to add
        height=None,  # Height attribute
        relative=None,  # Tuple of (position,px)
        link=False,   # Hyperlink to this image
        **kwargs):
    "A qmd image"
    kwargs,style = kwargs or {}, style or {}
    if height: kwargs["height"]= f"{height}px"
    if relative:
        pos,px = relative
        style["position"] = "relative"
        style[pos] = f"{px}px"
    res = meta(f'![]({fname})', classes=classes, style=style, **kwargs)
    return  f'[{res}]({fname})' if link else res

In [ ]:
#| export
def btn(txt, # Button text
        link,  # Button link URL
        classes=None,  # List of CSS classes to add
        style=None,    # Dict of CSS styles to add
        **kwargs):
    "A qmd button"
    return meta(f'[{txt}]({link})', classes=classes, style=style, role="button")

In [ ]:
#| export
def tbl_row(cols:list,  # Auto-stringified columns to show in the row
           ):
    "Create a markdown table row from `cols`"
    return '|' + '|'.join(str(c or '') for c in cols) + '|'

In [ ]:
#| export
def tbl_sep(sizes:int|list=3  # List of column sizes, or single `int` if all sizes the same
           ):
    "Create a markdown table separator with relative column size `sizes`"
    if isinstance(sizes,int): sizes = [3]*sizes
    return tbl_row('-'*s for s in sizes)

In [ ]:
#| export
def _install_nbdev():
    return div('''#### pip

```sh
pip install -U nbdev
```

#### conda

```sh
conda install -c fastai nbdev
```
''', ['panel-tabset'])

## Export -

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()